In [1]:
# # create empty DataFrame df and append row
# my_cols = ['date', 'days_lookback', 'syms_freq', 'symbols']
# # Creating Empty DataFrame and Storing it in variable df
# df_model_top_picks = pd.DataFrame(columns=my_cols)
# df_model_top_picks

In [2]:
import pandas as pd
from myUtils import pickle_load, pickle_dump
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.width', 900)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_picks  = f'df_picks'
fp_df_model_top_picks = f'df_model_top_picks'

verbose = True
# verbose = False

In [3]:
df = pickle_load(path_data_dump, fp_df_picks)
df = df.drop_duplicates(subset=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], keep='last')
df = df.sort_values(by=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], ascending=False)
df = df.reset_index(drop=True)
pickle_dump(df, path_data_dump, fp_df_picks)
print(f'len(df): {len(df)}')
print(df.head())

len(df): 192
  date_end_df_train  max_days_lookbacks     days_lookbacks sym_freq_15 sym_freq_14 sym_freq_13 sym_freq_12 sym_freq_11 sym_freq_10 sym_freq_9       sym_freq_8                     sym_freq_7              sym_freq_6                     sym_freq_5                     sym_freq_4                     sym_freq_3 sym_freq_2
0        2023-08-08                 120      [30, 60, 120]          []          []          []          []          []          []         []  ['SHV', 'STRL']                       ['SMCI']   ['BCC', 'LOB', 'TDS']        ['CLS', 'FTSM', 'SKYW']  ['BKU', 'EME', 'GBX', 'KDN...  ['AMPH', 'BZH', 'CVBF', 'E...         []
1        2023-08-08                 120  [15, 30, 60, 120]          []          []          []          []    ['STRL']     ['SHV']    ['TDS']               []  ['CLS', 'LOB', 'SATS', 'SM...          ['BCC', 'EME']  ['CORT', 'FFWM', 'FTSM', '...  ['BKU', 'DINO', 'GBX', 'KD...                ['AMPH', 'BZH']         []
2        2023-08-07              

In [4]:
df.head(50)

,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2023-08-08,120,"[30, 60, 120]",[],[],[],[],[],[],[],"['SHV', 'STRL']",['SMCI'],"['BCC', 'LOB', 'TDS']","['CLS', 'FTSM', 'SKYW']","['BKU', 'EME', 'GBX', 'KDN...","['AMPH', 'BZH', 'CVBF', 'E...",[]
1,2023-08-08,120,"[15, 30, 60, 120]",[],[],[],[],['STRL'],['SHV'],['TDS'],[],"['CLS', 'LOB', 'SATS', 'SM...","['BCC', 'EME']","['CORT', 'FFWM', 'FTSM', '...","['BKU', 'DINO', 'GBX', 'KD...","['AMPH', 'BZH']",[]
2,2023-08-07,120,"[30, 60, 120]",[],[],[],[],[],[],[],['SHV'],['SMCI'],['LOB'],"['BCC', 'CLS', 'EME', 'FTS...","['GBX', 'SATS', 'VNO']","['AVGO', 'BKU', 'BLDR', 'B...",[]
3,2023-08-07,120,"[15, 30, 60, 120]",[],[],[],[],[],['SHV'],[],['LOB'],"['CLS', 'SMCI']","['EME', 'SATS', 'TDS']","['BCC', 'CORT', 'CVBF', 'F...","['DINO', 'EDU', 'FFWM', 'G...",['AVGO'],[]
4,2023-08-04,120,"[30, 60, 120]",[],[],[],[],[],[],[],['SHV'],['SMCI'],['LOB'],"['BCC', 'CLS', 'FTSM', 'GB...","['EME', 'TDS', 'VNO']","['AMPH', 'AVGO', 'BKU', 'B...",[]
5,2023-08-04,120,"[15, 30, 60, 120]",[],[],[],[],[],['SHV'],[],['LOB'],"['CLS', 'SMCI', 'TDS']",[],"['BCC', 'CORT', 'CVBF', 'E...","['BKU', 'EDU', 'FFWM']",['AMPH'],[]
6,2023-08-03,120,"[30, 60, 120]",[],[],[],[],[],[],[],['SHV'],[],"['BCC', 'LOB', 'SMCI']","['FTSM', 'GBX', 'SKYW']","['AMPH', 'BZH', 'CLS', 'SL...","['AVGO', 'CVBF', 'ELF', 'G...",[]
7,2023-08-03,120,"[15, 30, 60, 120]",[],[],[],[],[],['SHV'],[],['LOB'],[],"['BCC', 'CLS', 'SMCI', 'WAL']","['CORT', 'CVBF', 'FTSM', '...","['AMPH', 'BZH', 'MATX', 'P...","['AVGO', 'CADE']",[]
8,2023-08-02,120,"[30, 60, 120]",[],[],[],[],[],[],[],['SHV'],['SMCI'],"['BZH', 'GBX', 'LOB']","['BCC', 'CLS', 'FTSM', 'SK...","['AMPH', 'EME', 'JOE']","['AVGO', 'CHTR', 'CSL', 'E...",[]
9,2023-08-02,120,"[15, 30, 60, 120]",[],[],[],[],[],['SHV'],['LOB'],[],"['CLS', 'SMCI']","['BZH', 'EME', 'GBX']","['BCC', 'FTSM', 'HP', 'JOE...","['AMPH', 'CVBF', 'PKX', 'V...","['ASTE', 'AVGO', 'BKI', 'B...",[]


In [5]:
df_mp_res = pickle_load(path_data_dump, 'df_model_picks_results')
df_mp = pickle_load(path_data_dump, 'df_model_picks')
# df_mp_res 
df_mp

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy,date_sell,sh_portf_sell,$_portf_sell,%_portf_chg,sh_SPY_sell,$_SPY_sell,%_SPY_chg,dif_%_chg,win
0,2023-07-14,"[30, 60, 120]",sym_freq_8,['SHV'],None,None,NaN,None,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,0
1,2023-07-13,"[30, 60, 120]",sym_freq_8,"['SHV', 'SKYW']",2023-07-14,"[4.0, 12.0]",926.000000,[2.0],898.559998,None,None,NaN,NaN,None,NaN,NaN,NaN,0
2,2023-07-12,"[30, 60, 120]",sym_freq_8,"['SHV', 'SKYW']",2023-07-13,"[4.0, 12.0]",938.440002,[2.0],899.119995,None,None,NaN,NaN,None,NaN,NaN,NaN,0
3,2023-07-11,"[30, 60, 120]",sym_freq_8,"['SHV', 'SKYW']",2023-07-12,"[4.0, 12.0]",933.999985,[2.0],892.039978,None,None,NaN,NaN,None,NaN,NaN,NaN,0


In [6]:
from itertools import product
from ast import literal_eval

my_cols = ['sym_freq_12', 'sym_freq_9', 'sym_freq_8']
my_days_lookbacks = [[30, 60, 120], [15, 30, 60, 120]]

my_dates = []
for my_days_lookback in my_days_lookbacks:
  my_row = (df['days_lookbacks'] == str(my_days_lookback))
  l_dates = df.loc[my_row, 'date_end_df_train'].tolist()     
  my_dates.append(l_dates)

my_dates = [val for sublist in my_dates for val in sublist]
my_dates = sorted(list(set(my_dates)), reverse=True)

d_lbk_freq = list(product(my_dates, my_days_lookbacks, my_cols))

picks = []
prev_date = ''
# print(f'Model predicts the best performing symbols are:\n[30, 60, 120]        sym_freq_9\n[15, 30, 60, 120]    sym_freq_12\n[30, 60, 120]        sym_freq_8')
print(f'Model predicts the best performing symbols are:')
print(f'1st    {"[30, 60, 120]":<20}{"sym_freq_9":<14}{"days_eval = 4":<20}')
print(f'2nd    {"[15, 30, 60, 120]":<20}{"sym_freq_12":<14}{"days_eval = 5":<20}')
print(f'3rd    {"[30, 60, 120]":<20}{"sym_freq_8":<14}{"days_eval = 4":<20}')
for i, item in enumerate(d_lbk_freq):
  _date, my_days_lookback, my_col = item[0], item[1], item[2]
  my_row = (df['date_end_df_train'] == _date) & (df['days_lookbacks'] == str(my_days_lookback))  
  list_in_df = df.loc[my_row, my_col]
  my_str = list_in_df.tolist()[0]
  my_list = literal_eval(my_str)
  blank = ' '
  if _date != prev_date:
    if prev_date == '':
      print('\nModel Results:')  
      print(f'{i:<6}{_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')
    else:
      print(f'\n{i:<6}{_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}') 
  else:
    print(f'{i:<6}{blank:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')

  picks.append((i, my_list))
  prev_date = _date

Model predicts the best performing symbols are:
1st    [30, 60, 120]       sym_freq_9    days_eval = 4       
2nd    [15, 30, 60, 120]   sym_freq_12   days_eval = 5       
3rd    [30, 60, 120]       sym_freq_8    days_eval = 4       

Model Results:
0     2023-08-08    [30, 60, 120]        sym_freq_12: []
1                   [30, 60, 120]        sym_freq_9 : []
2                   [30, 60, 120]        sym_freq_8 : ['SHV', 'STRL']
3                   [15, 30, 60, 120]    sym_freq_12: []
4                   [15, 30, 60, 120]    sym_freq_9 : ['TDS']
5                   [15, 30, 60, 120]    sym_freq_8 : []

6     2023-08-07    [30, 60, 120]        sym_freq_12: []
7                   [30, 60, 120]        sym_freq_9 : []
8                   [30, 60, 120]        sym_freq_8 : ['SHV']
9                   [15, 30, 60, 120]    sym_freq_12: []
10                  [15, 30, 60, 120]    sym_freq_9 : []
11                  [15, 30, 60, 120]    sym_freq_8 : ['LOB']

12    2023-08-04    [30, 60, 120]   

In [7]:
model_top_picks = [([30, 60, 120], 'sym_freq_9'), ([15, 30, 60, 120], 'sym_freq_12'), ([30, 60, 120], 'sym_freq_8')]

In [8]:
df_model_top_picks = pickle_load(path_data_dump, fp_df_model_top_picks)

z_items   = zip(d_lbk_freq, picks)
for i, item in enumerate(z_items):
  _date, _days_lookback, _sym_freq, _syms = item[0][0], item[0][1], item[0][2], item[1][1]
  print(i, _date, _days_lookback, _sym_freq, _syms)
  dlb_sym_freq = (_days_lookback, _sym_freq)
  if dlb_sym_freq in model_top_picks and _syms:
    row_add = [_date, _days_lookback, _sym_freq, _syms]
    df_model_top_picks.loc[len(df_model_top_picks)] = row_add
    print(f'added row to df_model_top_picks:\n{row_add}\n')


0 2023-08-08 [30, 60, 120] sym_freq_12 []
1 2023-08-08 [30, 60, 120] sym_freq_9 []
2 2023-08-08 [30, 60, 120] sym_freq_8 ['SHV', 'STRL']
added row to df_model_top_picks:
['2023-08-08', [30, 60, 120], 'sym_freq_8', ['SHV', 'STRL']]

3 2023-08-08 [15, 30, 60, 120] sym_freq_12 []
4 2023-08-08 [15, 30, 60, 120] sym_freq_9 ['TDS']
5 2023-08-08 [15, 30, 60, 120] sym_freq_8 []
6 2023-08-07 [30, 60, 120] sym_freq_12 []
7 2023-08-07 [30, 60, 120] sym_freq_9 []
8 2023-08-07 [30, 60, 120] sym_freq_8 ['SHV']
added row to df_model_top_picks:
['2023-08-07', [30, 60, 120], 'sym_freq_8', ['SHV']]

9 2023-08-07 [15, 30, 60, 120] sym_freq_12 []
10 2023-08-07 [15, 30, 60, 120] sym_freq_9 []
11 2023-08-07 [15, 30, 60, 120] sym_freq_8 ['LOB']
12 2023-08-04 [30, 60, 120] sym_freq_12 []
13 2023-08-04 [30, 60, 120] sym_freq_9 []
14 2023-08-04 [30, 60, 120] sym_freq_8 ['SHV']
added row to df_model_top_picks:
['2023-08-04', [30, 60, 120], 'sym_freq_8', ['SHV']]

15 2023-08-04 [15, 30, 60, 120] sym_freq_12 []
16

In [9]:
df_model_top_picks = df_model_top_picks.astype(str)
df_model_top_picks = df_model_top_picks.drop_duplicates(keep='last')
df_model_top_picks = df_model_top_picks.sort_values(by=['date', 'days_lookback', 'syms_freq'], ascending=False)
df_model_top_picks = df_model_top_picks.reset_index(drop=True)
pickle_dump(df_model_top_picks, path_data_dump, fp_df_model_top_picks)
print(f'len(df_model_top_picks): {len(df_model_top_picks)}')
print(f'{fp_df_model_top_picks}:\n{df_model_top_picks}')

len(df_model_top_picks): 107
df_model_top_picks:
           date  days_lookback   syms_freq          symbols
0    2023-08-08  [30, 60, 120]  sym_freq_8  ['SHV', 'STRL']
1    2023-08-07  [30, 60, 120]  sym_freq_8          ['SHV']
2    2023-08-04  [30, 60, 120]  sym_freq_8          ['SHV']
3    2023-08-03  [30, 60, 120]  sym_freq_8          ['SHV']
4    2023-08-02  [30, 60, 120]  sym_freq_8          ['SHV']
..          ...            ...         ...              ...
102  2023-03-20  [30, 60, 120]  sym_freq_9  ['FTSM', 'SHV']
103  2023-03-17  [30, 60, 120]  sym_freq_9  ['FTSM', 'SHV']
104  2023-03-17  [30, 60, 120]  sym_freq_8           ['GE']
105  2023-03-16  [30, 60, 120]  sym_freq_9  ['FTSM', 'SHV']
106  2023-03-15  [30, 60, 120]  sym_freq_9  ['FTSM', 'SHV']

[107 rows x 4 columns]


In [10]:
df_mp = df_model_top_picks.copy()
df_close = pickle_load(path_data_dump, 'df_close_clean')

In [11]:
def get_NYSE_date_n(date, dates_NYSE, n, verbose=False):

  if date in dates_NYSE:
    idx_last = len(dates_NYSE) - 1  # last index of dates_NYSE
    idx_date = dates_NYSE.get_loc(date)  # index of date
    idx_date_n = idx_date + n  # index of date_n
    if verbose:
      print(f"date: {date} is in dates_NYSE,  date's position in dates_NYSE is: {idx_date} of {idx_last},  n: {n},  idx_date_n: {idx_date_n},")
    if 0 <= idx_date_n <= idx_last:  # idx_date_n is within bounds of dates_NYSE
      date_n = dates_NYSE[idx_date_n]
      if verbose:      
        print(f'idx_date_n: {idx_date_n} is wiithin bounds of dates_NYSE ("0 to {idx_last}), date_n: {date_n}"\n')
    else:  # idx_date_n is out-of-bounds of dates_NYSE
      date_n = None   
      if verbose:  
        print(f'idx_date_n: {idx_date_n} is out-of-bounds of dates_NYSE ("0 to {idx_last})"\n')
  else:  # date not in dates_NYSE
      date_n = None  
      if verbose:  
        print(f'date: {date} is not in dates_NYSE\n')

  return date_n  

In [12]:
def get_portf_buy(df_close, date, str_symbols, portf_target, verbose=False):
    import numpy as np
    from ast import literal_eval    
    l_syms = literal_eval(str_symbols)  # convert list stored as str back to list
    # array of closing prices corresponding to symbols in l_syms    
    ar_price = df_close.loc[date][l_syms].values  
    sym_cnt = len(l_syms)  # number of symbols
    amt_per_sym = portf_target / sym_cnt  # target dollar investment in each symbol
    ar_sym_share = np.floor(amt_per_sym / ar_price)  # array of shares for each symbol
    ar_sym_amt = ar_price * ar_sym_share  # array of actual dollar amount invested in each symbol
    portf_value = sum(ar_sym_amt)  # total actual portfolio value
    if verbose:
        # print(date, l_syms, ar_price, ar_sym_share, ar_sym_amt, portf_value)    
        print(f'{date = }, {l_syms = }, {ar_price = }, {ar_sym_share = }, {ar_sym_amt = }, {portf_value = }')            
        print(f'{date} {portf_value = }')
    return date, l_syms, ar_price, ar_sym_share, ar_sym_amt, portf_value

In [13]:
def get_SPY_buy(df_close, date, portf_target, symbol="SPY", verbose=False):   
    import numpy as np
    price = df_close.loc[date][symbol]    
    share = np.floor(portf_target / price)  # array of shares for each symbol
    value = price * share  # array of actual dollar amount invested in each symbol
    if verbose:
        # print(date, symbol, price, share, value)
        print(f'{date = }, {symbol = }, {price = }, {share = }, {value = }')                        
        print(f'{date} {symbol} {value = }')
    return date, price, share, value

In [14]:
import pandas_market_calendars as mcal
nyse = mcal.get_calendar('NYSE')
dates_NYSE_2013_2023 = nyse.valid_days(start_date='2013-01-01', end_date='2023-12-31').strftime('%Y-%m-%d')
print(f'len(dates_NYSE_2013_2023): {len(dates_NYSE_2013_2023)}')
print(f'dates_NYSE_2013_2023[0]:  {dates_NYSE_2013_2023[0]}, dates_NYSE_2013_2023[-1]: {dates_NYSE_2013_2023[-1]}')
print(f'{dates_NYSE_2013_2023}') 

len(dates_NYSE_2013_2023): 2768
dates_NYSE_2013_2023[0]:  2013-01-02, dates_NYSE_2013_2023[-1]: 2023-12-29
Index(['2013-01-02', '2013-01-03', '2013-01-04', '2013-01-07', '2013-01-08', '2013-01-09', '2013-01-10', '2013-01-11', '2013-01-14', '2013-01-15',
       ...
       '2023-12-15', '2023-12-18', '2023-12-19', '2023-12-20', '2023-12-21', '2023-12-22', '2023-12-26', '2023-12-27', '2023-12-28', '2023-12-29'], dtype='object', length=2768)


#### TODO: 2023-07-15 only evaluate performance of df_model_picks, after evaluation, append results to df_model_picks_results 


#### 2023-07-11 got KeyError: "['ISEE'] not in index". ISEE was in past model picks, but since then ISEE was acquired. The symbol was dropped from df_close_clean. Current program recalculate the performance of all the past picks. Rewrite program such that performance of past picks with win records are not recalculated.

In [15]:
import numpy as np
# from ast import literal_eval
z_date_syms = zip(df_mp.date, df_mp.symbols)

idx_close = df_close.index.strftime('%Y-%m-%d')

n = 1  # n=1 is the buy date
# n = 4  # n=4 is the sell date

date_buy = []  # buy date of portfolio
shares_syms = []  # lists of shares of each symbol brought on date_buy
value_portf = [] # list of porfolio value on date_buy
shares_SPY = []  # list of shares of SPY brought on date_buy
value_SPY = []  # list of value of SPY shares on date_buy 
for date, syms in z_date_syms:
    next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n, verbose=verbose)

    if next_date_n in idx_close:
        close_date_n = next_date_n
        print(f'date: {date},  n: {n},  next_date_n: {next_date_n} is in df_close')
        print(f'picks for {date}: {syms}')      
    else:
        close_date_n = None
        print(f'date: {date},  n: {n},  next_date_n: {next_date_n} is not in df_close')
        print(f'picks for {date}: {syms}')        

    if close_date_n is None:
        p_date = None
        p_l_syms = None
        p_ar_price = None
        p_ar_sym_share = None
        p_ar_sym_amt = None
        p_portf_value = None  # data not available in df_close
        
        s_date = None
        s_price = None
        s_share = None
        s_value = None  # data not available in df_close

        date_buy.append(p_date)
        shares_syms.append(p_l_syms)
        value_portf.append(p_portf_value)
        shares_SPY.append(s_share)
        value_SPY.append(s_value)    
        
        print(f"No data for next_date_n {next_date_n}, pick's portf value = None")
        print(f'No data for next_date_n {next_date_n}, SPY portf value =    None')

    else:    
        p_date, p_l_syms, p_ar_price, p_ar_sym_share, p_ar_sym_amt, p_portf_value = \
            get_portf_buy(df_close, next_date_n, syms, portf_target=1000, verbose=verbose)
        s_date, s_price, s_share, s_value = \
            get_SPY_buy(df_close, next_date_n, portf_target=1000, verbose=verbose)
                
        date_buy.append(p_date)
        shares_syms.append(p_ar_sym_share)
        value_portf.append(p_portf_value)
        shares_SPY.append(s_share)
        value_SPY.append(s_value)

        print(f"next_date_n pick's portf value = {p_portf_value}")
        print(f'next_date_n SPY portf value =    {s_value}')

    print('='*20, '\n')       
    

date: 2023-08-08 is in dates_NYSE,  date's position in dates_NYSE is: 2667 of 2767,  n: 1,  idx_date_n: 2668,
idx_date_n: 2668 is wiithin bounds of dates_NYSE ("0 to 2767), date_n: 2023-08-09"

date: 2023-08-08,  n: 1,  next_date_n: 2023-08-09 is not in df_close
picks for 2023-08-08: ['SHV', 'STRL']
No data for next_date_n 2023-08-09, pick's portf value = None
No data for next_date_n 2023-08-09, SPY portf value =    None

date: 2023-08-07 is in dates_NYSE,  date's position in dates_NYSE is: 2666 of 2767,  n: 1,  idx_date_n: 2667,
idx_date_n: 2667 is wiithin bounds of dates_NYSE ("0 to 2767), date_n: 2023-08-08"

date: 2023-08-07,  n: 1,  next_date_n: 2023-08-08 is in df_close
picks for 2023-08-07: ['SHV']
date = '2023-08-08', l_syms = ['SHV'], ar_price = array([110.09999847]), ar_sym_share = array([9.]), ar_sym_amt = array([990.89998627]), portf_value = 990.8999862670898
2023-08-08 portf_value = 990.8999862670898
date = '2023-08-08', symbol = 'SPY', price = 448.75, share = 2.0, value =

#### Below cells calculate past performance of model picks

In [16]:
def calc_portf_shares(df_close, date, str_symbols, portf_target):
  # calculate number of shares to buy for symbols in str_symbols to meet port_target
  import numpy as np
  from ast import literal_eval    
  l_syms = literal_eval(str_symbols)  # convert list stored as str back to list
  # array of closing prices corresponding to symbols in l_syms    
  ar_price = df_close.loc[date][l_syms].values  
  sym_cnt = len(l_syms)  # number of symbols
  amt_per_sym = portf_target / sym_cnt  # target dollar investment in each symbol
  ar_shares = np.floor(amt_per_sym / ar_price)  # array of shares for each symbol
  return ar_shares

In [17]:
def calc_portf_value(df_close, date, str_symbols, ar_shares, verbose=False):
    import numpy as np
    from ast import literal_eval    
    l_syms = literal_eval(str_symbols)  # convert list stored as str back to list
    # array of closing prices corresponding to symbols in l_syms    
    ar_price = df_close.loc[date][l_syms].values  
    ar_value = ar_price * ar_shares  # array of actual dollar amount invested in each symbol
    portf_value = sum(ar_value)  # total actual portfolio value
    if verbose:
        print(f'{date = }, {l_syms = }, {ar_price = }, {ar_shares = }, {ar_value = }, {portf_value = }')            
        print(f'{date} {portf_value = }')
    return date, l_syms, ar_price, ar_shares, ar_value, portf_value

In [18]:
def get_next_date_n(df_close, date, n=1):

    """Get the next n(default=1) date from df_close. date has to in df_close.index.

    Edge case: The function won't return the first date in df_close as the
    next trading date, if date is a trading date just before the first date in df_close.
    """

    len_df_close = len(df_close)
    idx_dates = df_close.index.strftime('%Y-%m-%d')  # date index of df_close    

    if date in idx_dates:
        idx_next_date = idx_dates.get_loc(date) + n  # get index position of next date
        print(f'n: {n},  idx_next_date: {idx_next_date}')

        if idx_next_date >= 0:  # if idx_nex_date is negative, stop index loop back from index[0] to index[-1]    
            if idx_next_date >= (len_df_close - n):  # next date's index is within bound of df_close
                next_date = idx_dates[idx_dates.get_loc(date) + n]  # get the next date
                return next_date
        
              
            else:  # idx_next_date is out-of_bounds df_close
                print(f"{date} is the last date in df_close. No data for next {n} day")
                return None    
        
        
        else:  # idx_next_date is negative, no data in df_close
            # no data from df_close
            print(f'idx_next_date: {idx_next_date} is negative, no data for {n} days from {date} in df_close')

    else:     
        # no data from df_close
        print(f'no data for {date} in df_close')
        return None

In [19]:
def is_date_in_close(date, df_close):
  idx_close = df_close.index.strftime('%Y-%m-%d')
  if date in idx_close:
    return date
  else:
    return None

In [20]:
def calc_portf_value_date_buy(my_dates, my_symbols, df_close, portf_target, n, verbose=False):
  
  z_date_syms = zip(my_dates, my_symbols)

  date_exec = []  # buy date of portfolio
  shares_syms = []  # lists of shares of each symbol brought on date
  value_portf = [] # list of porfolio value on date
  shares_SPY = []  # list of shares of SPY brought on date
  value_SPY = []  # list of value of SPY shares on date 

  for date, syms in z_date_syms:
    next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n, verbose=False)
    close_date_n = is_date_in_close(next_date_n, df_close)

    if close_date_n is None:
      p_date = None
      p_ar_shares = None
      p_portf_value = None  # set to None when data are not available in df_close
      SPY_shares = None
      SPY_value = None  # set to None when data are not available in df_close

      if verbose:
        print(f"No data for close_date_n {close_date_n}, pick's portf value = None")
        print(f'No data for close_date_n {close_date_n}, SPY portf value =    None')

    else:    
      p_ar_shares = calc_portf_shares(df_close, close_date_n, syms, portf_target)
      p_date, l_syms, ar_price, ar_shares, ar_value, p_portf_value = \
        calc_portf_value(df_close, close_date_n, syms, p_ar_shares, verbose)

      syms = str(['SPY'])
      SPY_shares = calc_portf_shares(df_close, close_date_n, syms, portf_target)
      date, l_syms, ar_price, ar_shares, ar_value, SPY_value = \
        calc_portf_value(df_close, close_date_n, syms, SPY_shares, verbose)

      if verbose:
        print(f"close_date_n pick's portf value = {p_portf_value}")
        print(f'close_date_n SPY portf value =    {SPY_value}')

    date_exec.append(p_date)
    shares_syms.append(p_ar_shares)
    value_portf.append(p_portf_value)
    shares_SPY.append(SPY_shares)
    value_SPY.append(SPY_value)

    print('='*20, '\n')

  return date_exec, shares_syms, value_portf, shares_SPY, value_SPY       
    

In [21]:
def calc_portf_value_date_n(my_dates, my_symbols, df_close, my_portf_shares, my_SPY_shares, n, verbose=False):
  
  z_date_syms_shares = zip(my_dates, my_symbols, my_portf_shares, my_SPY_shares)

  date_exec = []  # buy date of portfolio
  shares_syms = []  # lists of shares of each symbol brought on date
  value_portf = [] # list of porfolio value on date
  shares_SPY = []  # list of shares of SPY brought on date
  value_SPY = []  # list of value of SPY shares on date 

  for date, symbols, portf_shares, SPY_shares in z_date_syms_shares:
    next_date_n = get_NYSE_date_n(date, dates_NYSE_2013_2023, n, verbose=False)
    close_date_n = is_date_in_close(next_date_n, df_close)

    if close_date_n is None:
      p_date_exec = None
      p_ar_shares = None
      p_value_portf = None  # set to None when data are not available in df_close
      SPY_ar_shares = None
      SPY_value_portf = None # set to None when data are not available in df_close

      if verbose:
        print(f"No data for close_date_n {close_date_n}, pick's portf value = None")
        # print(f'No data for next_date_n {next_date_n}, SPY portf value =    None')

    else: 
      if portf_shares is None:
        p_date_exec = None
        p_ar_shares = None
        p_value_portf = None
        SPY_ar_shares = None
        SPY_value_portf = None

      else:  
        p_date_exec, p_ar_syms, p_ar_price, p_ar_shares, p_ar_value, p_value_portf = \
          calc_portf_value(df_close, close_date_n, symbols, portf_shares, verbose)
        
        SPY = str(['SPY'])
        SPY_date_exec, SPY_ar_syms, SPY_ar_price, SPY_ar_shares, SPY_ar_value, SPY_value_portf = \
          calc_portf_value(df_close, close_date_n, SPY, SPY_shares, verbose) 

        if verbose:
          print(f"next_date_n pick's portf value = {p_value_portf}")
          print(f'next_date_n SPY portf value =    {SPY_value_portf}')

    date_exec.append(p_date_exec)
    shares_syms.append(p_ar_shares)
    value_portf.append(p_value_portf)
    shares_SPY.append(SPY_ar_shares)
    value_SPY.append(SPY_value_portf)

    print('='*20, '\n')

  return date_exec, shares_syms, value_portf, shares_SPY, value_SPY  

In [22]:
# total target investment for each day's picks, if day's pick has 2 symbols, the function will try to invest $500 for each symbol
portf_target = 1000  
date_buy, shares_syms, value_portf, shares_SPY, value_SPY = calc_portf_value_date_buy(df_mp.date, df_mp.symbols, df_close, portf_target, n=1, verbose=verbose)

No data for close_date_n None, pick's portf value = None
No data for close_date_n None, SPY portf value =    None

date = '2023-08-08', l_syms = ['SHV'], ar_price = array([110.09999847]), ar_shares = array([9.]), ar_value = array([990.89998627]), portf_value = 990.8999862670898
2023-08-08 portf_value = 990.8999862670898
date = '2023-08-08', l_syms = ['SPY'], ar_price = array([448.75]), ar_shares = array([2.]), ar_value = array([897.5]), portf_value = 897.5
2023-08-08 portf_value = 897.5
close_date_n pick's portf value = 990.8999862670898
close_date_n SPY portf value =    897.5

date = '2023-08-07', l_syms = ['SHV'], ar_price = array([110.08000183]), ar_shares = array([9.]), ar_value = array([990.72001648]), portf_value = 990.7200164794922
2023-08-07 portf_value = 990.7200164794922
date = '2023-08-07', l_syms = ['SPY'], ar_price = array([450.70999146]), ar_shares = array([2.]), ar_value = array([901.41998291]), portf_value = 901.4199829101562
2023-08-07 portf_value = 901.4199829101562
c

In [23]:
df_mp['date_buy'] = date_buy
df_mp['sh_portf_buy'] = shares_syms
df_mp['$_portf_buy'] = value_portf
df_mp['sh_SPY_buy'] = shares_SPY
df_mp['$_SPY_buy'] = value_SPY
# df_mp.tail()
# df_mp.head()
df_mp

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy
0,2023-08-08,"[30, 60, 120]",sym_freq_8,"['SHV', 'STRL']",None,None,NaN,None,NaN
1,2023-08-07,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-08,[9.0],990.899986,[2.0],897.500000
2,2023-08-04,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-07,[9.0],990.720016,[2.0],901.419983
3,2023-08-03,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-04,[9.0],990.720016,[2.0],893.619995
4,2023-08-02,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-03,[9.0],990.450027,[2.0],897.679993
...,...,...,...,...,...,...,...,...,...
102,2023-03-20,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']",2023-03-21,"[8.0, 4.0]",900.636627,[2.0],794.867371
103,2023-03-17,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']",2023-03-20,"[8.0, 4.0]",901.225372,[2.0],784.565613
104,2023-03-17,"[30, 60, 120]",sym_freq_8,['GE'],2023-03-20,[11.0],988.389091,[2.0],784.565613
105,2023-03-16,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']",2023-03-17,"[8.0, 4.0]",900.597229,[2.0],777.093384


In [24]:
date_exec, shares_syms, value_portf, shares_SPY, value_SPY  = \
  calc_portf_value_date_n(df_mp.date, df_mp.symbols, df_close, df_mp.sh_portf_buy, df_mp.sh_SPY_buy, n=4, verbose=verbose)

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

No data for close_date_n None, pick's portf value = None

date = '2023-08-08', l_syms = ['SHV'], ar_price = array([110.09999847]), ar_shares = array([9.]), ar_value = array([990.89998627]), portf_value = 990.8999862670898
2023-08-08 portf_value = 990.8999862670898
date = '2023-08-08', l_syms = ['SPY'], ar_price = array([448.75]), ar_shares = array([2.]), ar_value = array([897.5]), portf_value = 897.5
2023-08-08 portf_value = 897.5
next_date_n pick's portf value = 990.8999862670898
next_date_n SPY portf value =    897.5

date = '2023-08-02', l_syms = ['SHV'], ar_price = array([110.01000214]), ar_shares = array([9.]), ar_value = array([990.09001923]), portf_value = 990.0900192260742
2023-08-02 portf_value = 990.0900192260742
date = '2023-08-02', l_syms = ['SPY'], ar_price = array([450.13000488]), ar_shares = array([2

In [25]:
df_mp['date_sell'] = date_exec
df_mp['sh_portf_sell'] = shares_syms
df_mp['$_portf_sell'] = value_portf
df_mp['%_portf_chg'] = (df_mp['$_portf_sell'] / df_mp['$_portf_buy'] - 1) * 100

df_mp['sh_SPY_sell'] = shares_SPY
df_mp['$_SPY_sell'] = value_SPY
df_mp['%_SPY_chg'] = (df_mp['$_SPY_sell'] / df_mp['$_SPY_buy'] - 1) * 100

df_mp['dif_%_chg'] = df_mp['%_portf_chg'] - df_mp['%_SPY_chg']

In [26]:
# get symbol's close price for that date
date = '2023-03-16'
# _date = '2023-03-21'
# _sym = 'GE'
# _sym = 'NVDA'
_sym = 'FTSM'
# _sym = 'SHV'
_sym = 'SPY'
# _c = df_close.loc['2023-04-11']['NVDA']
_c = df_close.loc[_date][_sym]
_c

386.3647766113281

#### dif_%_chg is the percentage change of model icks - percentage change of SPY

In [27]:
df_mp.head(20)

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy,date_sell,sh_portf_sell,$_portf_sell,%_portf_chg,sh_SPY_sell,$_SPY_sell,%_SPY_chg,dif_%_chg
0,2023-08-08,"[30, 60, 120]",sym_freq_8,"['SHV', 'STRL']",None,None,NaN,None,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN
1,2023-08-07,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-08,[9.0],990.899986,[2.0],897.500000,None,None,NaN,NaN,None,NaN,NaN,NaN
2,2023-08-04,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-07,[9.0],990.720016,[2.0],901.419983,None,None,NaN,NaN,None,NaN,NaN,NaN
3,2023-08-03,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-04,[9.0],990.720016,[2.0],893.619995,None,None,NaN,NaN,None,NaN,NaN,NaN
4,2023-08-02,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-03,[9.0],990.450027,[2.0],897.679993,2023-08-08,[9.0],990.899986,0.045430,[2.0],897.500000,-0.020051,0.065481
5,2023-07-27,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-28,[9.0],989.577782,[2.0],913.840027,2023-08-02,[9.0],990.090019,0.051763,[2.0],900.260010,-1.486039,1.537802
6,2023-07-26,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-27,[9.0],989.308891,[2.0],904.979980,2023-08-01,[9.0],989.909981,0.060759,[2.0],912.960022,0.881792,-0.821034
7,2023-07-25,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-26,[9.0],988.950394,[2.0],911.020020,2023-07-31,[9.0],989.756996,0.081561,[2.0],915.580017,0.500538,-0.418976
8,2023-07-21,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-24,[9.0],988.681572,[2.0],908.400024,2023-07-27,[9.0],989.308891,0.063450,[2.0],904.979980,-0.376491,0.439941
9,2023-07-20,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-21,[9.0],988.591965,[2.0],904.359985,2023-07-26,[9.0],988.950394,0.036257,[2.0],911.020020,0.736436,-0.700180


In [28]:
df_mp['win'] = np.where(df_mp['dif_%_chg']>0,1,0)
df_mp.head(20)

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy,date_sell,sh_portf_sell,$_portf_sell,%_portf_chg,sh_SPY_sell,$_SPY_sell,%_SPY_chg,dif_%_chg,win
0,2023-08-08,"[30, 60, 120]",sym_freq_8,"['SHV', 'STRL']",None,None,NaN,None,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,0
1,2023-08-07,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-08,[9.0],990.899986,[2.0],897.500000,None,None,NaN,NaN,None,NaN,NaN,NaN,0
2,2023-08-04,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-07,[9.0],990.720016,[2.0],901.419983,None,None,NaN,NaN,None,NaN,NaN,NaN,0
3,2023-08-03,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-04,[9.0],990.720016,[2.0],893.619995,None,None,NaN,NaN,None,NaN,NaN,NaN,0
4,2023-08-02,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-03,[9.0],990.450027,[2.0],897.679993,2023-08-08,[9.0],990.899986,0.045430,[2.0],897.500000,-0.020051,0.065481,1
5,2023-07-27,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-28,[9.0],989.577782,[2.0],913.840027,2023-08-02,[9.0],990.090019,0.051763,[2.0],900.260010,-1.486039,1.537802,1
6,2023-07-26,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-27,[9.0],989.308891,[2.0],904.979980,2023-08-01,[9.0],989.909981,0.060759,[2.0],912.960022,0.881792,-0.821034,0
7,2023-07-25,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-26,[9.0],988.950394,[2.0],911.020020,2023-07-31,[9.0],989.756996,0.081561,[2.0],915.580017,0.500538,-0.418976,0
8,2023-07-21,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-24,[9.0],988.681572,[2.0],908.400024,2023-07-27,[9.0],989.308891,0.063450,[2.0],904.979980,-0.376491,0.439941,1
9,2023-07-20,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-21,[9.0],988.591965,[2.0],904.359985,2023-07-26,[9.0],988.950394,0.036257,[2.0],911.020020,0.736436,-0.700180,0


### Model Performance Not Good

In [29]:
wins = df_mp['win'].sum()
attempts = len(df_mp['dif_%_chg'].dropna())

win_rate = wins / attempts
print(f'win_rate: {win_rate:0.6f}, wins: {wins}, attempts: {attempts}')
print(f'sum(dff_%_chg): {df_mp["dif_%_chg"].sum():0.6f}')


win_rate: 0.339806, wins: 35, attempts: 103
sum(dff_%_chg): -61.556613


In [30]:
df = df_mp.copy()

In [31]:
df_model_picks = df[df['dif_%_chg'].isnull()].copy()
pickle_dump(df_model_picks, path_data_dump, 'df_model_picks')
_df = pickle_load(path_data_dump, 'df_model_picks')
_df

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy,date_sell,sh_portf_sell,$_portf_sell,%_portf_chg,sh_SPY_sell,$_SPY_sell,%_SPY_chg,dif_%_chg,win
0,2023-08-08,"[30, 60, 120]",sym_freq_8,"['SHV', 'STRL']",None,None,NaN,None,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,0
1,2023-08-07,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-08,[9.0],990.899986,[2.0],897.500000,None,None,NaN,NaN,None,NaN,NaN,NaN,0
2,2023-08-04,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-07,[9.0],990.720016,[2.0],901.419983,None,None,NaN,NaN,None,NaN,NaN,NaN,0
3,2023-08-03,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-04,[9.0],990.720016,[2.0],893.619995,None,None,NaN,NaN,None,NaN,NaN,NaN,0


In [32]:
idx_last_row = df_model_picks.index[-1]
df_model_picks_results = df.iloc[(idx_last_row + 1)::].copy()
pickle_dump(df_model_picks_results, path_data_dump, 'df_model_picks_results')
_df = pickle_load(path_data_dump, 'df_model_picks_results')
_df

,date,days_lookback,syms_freq,symbols,date_buy,sh_portf_buy,$_portf_buy,sh_SPY_buy,$_SPY_buy,date_sell,sh_portf_sell,$_portf_sell,%_portf_chg,sh_SPY_sell,$_SPY_sell,%_SPY_chg,dif_%_chg,win
4,2023-08-02,"[30, 60, 120]",sym_freq_8,['SHV'],2023-08-03,[9.0],990.450027,[2.0],897.679993,2023-08-08,[9.0],990.899986,0.045430,[2.0],897.500000,-0.020051,0.065481,1
5,2023-07-27,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-28,[9.0],989.577782,[2.0],913.840027,2023-08-02,[9.0],990.090019,0.051763,[2.0],900.260010,-1.486039,1.537802,1
6,2023-07-26,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-27,[9.0],989.308891,[2.0],904.979980,2023-08-01,[9.0],989.909981,0.060759,[2.0],912.960022,0.881792,-0.821034,0
7,2023-07-25,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-26,[9.0],988.950394,[2.0],911.020020,2023-07-31,[9.0],989.756996,0.081561,[2.0],915.580017,0.500538,-0.418976,0
8,2023-07-21,"[30, 60, 120]",sym_freq_8,['SHV'],2023-07-24,[9.0],988.681572,[2.0],908.400024,2023-07-27,[9.0],989.308891,0.063450,[2.0],904.979980,-0.376491,0.439941,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,2023-03-20,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']",2023-03-21,"[8.0, 4.0]",900.636627,[2.0],794.867371,2023-03-24,"[8.0, 4.0]",901.892395,0.139431,[2.0],788.570801,-0.792154,0.931585,1
103,2023-03-17,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']",2023-03-20,"[8.0, 4.0]",901.225372,[2.0],784.565613,2023-03-23,"[8.0, 4.0]",901.617645,0.043527,[2.0],783.429871,-0.144761,0.188287,1
104,2023-03-17,"[30, 60, 120]",sym_freq_8,['GE'],2023-03-20,[11.0],988.389091,[2.0],784.565613,2023-03-23,[11.0],1006.745476,1.857202,[2.0],783.429871,-0.144761,2.001963,1
105,2023-03-16,"[30, 60, 120]",sym_freq_9,"['FTSM', 'SHV']",2023-03-17,"[8.0, 4.0]",900.597229,[2.0],777.093384,2023-03-22,"[8.0, 4.0]",901.264618,0.074105,[2.0],781.317688,0.543603,-0.469498,0
